# Chronométrage avancé

Ce *notebook* a pour but de proposer des éléments approfondis à propos du chronométrage du processus de génération de features géométriques dans un nuage de points 3D.

Seront étudiés les points suivants :
- les métriques d'accumulation, ou `pandas.DataFrame.query` *vs* `pandas.DataFrame.loc`
- le KDtree, ou `sklearn.neighbors.KDTree` *vs* `scipy.spatial.cKDTree`

## Introduction

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import seaborn as sns

%matplotlib inline

In [2]:
TIMER_BASE_FILE = Path("..", "data", "profiling", "base", "timers", "timers.csv")
TIMER_QUERY_FILE = Path("..", "data", "profiling", "query", "timers", "timers.csv")
TIMER_IDX_FILE = Path("..", "data", "profiling", "accidx", "timers", "timers.csv")
TIMER_SCIPYTREE_FILE = Path("..", "data", "profiling", "scipytree", "timers", "timers.csv")

In [3]:
timers_base = pd.read_csv(TIMER_BASE_FILE)
timers_query = pd.read_csv(TIMER_QUERY_FILE)
timers_idx = pd.read_csv(TIMER_IDX_FILE)
timers_scipytree = pd.read_csv(TIMER_SCIPYTREE_FILE)

In [4]:
def extract(df, function=None, nb_points=None, nb_neighbors=None, feature_set=None):
    """Extraction des données relatives à la fonction 'retrieve_accumulation_features'
    
    Parameters
    ----------
    df : pandas.DataFrame
        timers
    function : str
         Fonction d'intérêt
    nb_points : int
         Nombre de points sur lequel faire la comparaison
    nb_neighbors : int
         Nombre de voisins sur lequel faire la comparaison
    feature_set : str
         Ensemble de feature sur lequel faire la comparaison
    Returns
    -------
    pandas.DataFrame
        sous-ensemble d'enregistrements spécifiques à la fonction identifiée
    """
    df_extract = df.copy()
    if function is not None:
        df_extract = df_extract.loc[[function in c for c in df_extract["function"]]]
    if nb_points is not None:
        df_extract = df_extract.loc[df["nb_points"] == nb_points]
    if nb_neighbors is not None:
        df_extract = df_extract.loc[df["nb_neighbors"] == nb_neighbors]
    if feature_set is not None:
        df_extract = df_extract.loc[df["feature_set"] == feature_set]
    df_extract.drop(["nb_neighbors", "feature_set", "nb_calls"], axis=1, inplace=True)
    return df_extract

In [5]:
def compute_ratio(df1, df2):
    """Compute ratio of performances between two scenarios
    
    Parameters
    ----------
    df1 : pandas.DataFrame
        First scenario performances
    df2 : pandas.DataFrame
        Second scenario performances
    
    Returns
    -------
    pandas.DataFrame
        Compared performances
    """
    df1_ = df1.set_index("nb_points").copy()
    df2_ = df2.set_index("nb_points").copy()
    df = pd.concat([df1_[["cum_time"]], df2_[["cum_time"]]], axis=1)
    df.columns = ["cum_time_1", "cum_time_2"]
    df["ratio"] = df["cum_time_1"] / df["cum_time_2"]
    return df

## Métriques d'accumulation

Cette section va permettre d'étudier un peu plus en détail la fonction `retrieve_accumulation_features`. Cette fonction est la plus longue du processus de calcul des features géométriques dans un nuage de point 3D. Elle était initialement implémentée à l'aide de la méthode `pandas.DataFrame.query(.)`. Nous proposons ici de comparer cette implémentation avec l'accesseur plus classique `pandas.DataFrame.loc[.]`.

In [6]:
retrieve_base = extract(timers_base, function="retrieve", nb_neighbors=50, feature_set="full")

In [7]:
retrieve_query = extract(timers_query, function="retrieve", nb_neighbors=50, feature_set="full")

In [8]:
compute_ratio(retrieve_base, retrieve_query)

,cum_time_1,cum_time_2,ratio
nb_points,,,
50000,168.319060,275.738380,0.610430
5000,16.287677,25.943182,0.627821
100000,378.940976,589.922543,0.642357
1000,3.083841,4.828300,0.638701
10000,33.095040,51.726613,0.639807


Au vu de ces chiffres, il s'avère que l'accesseur `pandas.DataFrame.loc[.]` est plus rapide que la méthode `pandas.DataFrame.query(.)`, et permet une réduction du temps de calcul d'environ 35 à 40%.

### Accession directe

Il s'avère en fait que la méthode `accumulation_2d_neighborhood(.)` préserve l'ordre des points, et même plus, inclut les coordonnées des points du nuage. Il suffit ainsi d'itérer sur cette structure plutôt que sur le nuage de point simple.

In [9]:
retrieve_full_base = extract(timers_base, function="all_features", nb_neighbors=50, feature_set="full")

In [10]:
retrieve_full_idx = extract(timers_idx, function="all_features", nb_neighbors=50, feature_set="full")

In [11]:
compute_ratio(retrieve_full_idx, retrieve_full_base)

,cum_time_1,cum_time_2,ratio
nb_points,,,
1000,NaN,4.300485,NaN
5000,5.259709,22.618200,0.232543
10000,10.958623,46.072779,0.237855
50000,59.683575,233.397269,0.255717
100000,118.700419,518.431784,0.228961


L'accession directe via le nuage de point "amélioré" par les features d'accumulation est ainsi environ 6 fois plus rapide (seulement 16-17% du temps pris par la version avec l'accesseur `df.loc[.]`...

## KDTree

Cette seconde partie a pour but d'évaluer différentes fonctions pour le calcul du KDTree, et son requêtage pour reconstituer le voisinage de chaque point évalué. Deux options techniques sont ici comparées :
- le KDTree de `scikit-learn` ;
- le KDTree de `scipy` implémenté à partir de Cython (il existe aussi une version Python standard).

#### Construction du KDTree

In [12]:
skl_compute = extract(timers_base, function="compute_tree", nb_neighbors=50, feature_set="alphabeta")
scipy_compute = extract(timers_scipytree, function="compute_tree", nb_neighbors=50, feature_set="alphabeta")

In [13]:
compute_ratio(skl_compute, scipy_compute)

,cum_time_1,cum_time_2,ratio
nb_points,,,
1000,0.000183,0.000177,1.033898
5000,0.000627,0.000665,0.942857
10000,0.001347,0.002076,0.648844
50000,0.011829,0.013384,0.883816
100000,0.030741,0.032120,0.957067
500000,NaN,0.414903,NaN
1000000,NaN,1.396491,NaN


Les performances des deux KDTree sont assez proches l'une de l'autre, on note cependant que le KDTree de `scikit-learn` est légèrement plus rapide à construire que celui de `scipy`.

#### Requêtage du KDTree

In [14]:
skl_query = extract(timers_base, function="build_neighborhood", nb_neighbors=50, feature_set="alphabeta")
scipy_query = extract(timers_scipytree, function="build_neighborhood", nb_neighbors=50, feature_set="alphabeta")

In [15]:
compute_ratio(skl_query, scipy_query)

,cum_time_1,cum_time_2,ratio
nb_points,,,
1000,0.131815,0.112442,1.172293
5000,0.836286,0.570470,1.465960
10000,1.610228,1.153717,1.395687
50000,10.432209,6.676162,1.562606
100000,20.581425,14.000602,1.470039
500000,NaN,84.370041,NaN
1000000,NaN,172.671911,NaN


Au contraire de l'étape de construction, il semble que le requêtage est significativement plus lent avec `scikit-learn` : entre 17 et 56% de plus sur l'ensemble des itérations, et cette différence semble d'ailleurs s'accentuer avec le nombre de points considérés.